# SOC_est_1 로직 시각화
## 배터리 SOC 추정기 (State of Charge Estimator)

**LaTeX 다이어그램과 Python 시뮬레이션으로 SOC_est_1.mo의 로직을 분석합니다.**

---
*작성일: 2025년 8월 29일*
*Modelica 파일: SOC_est_1.mo*

## 1. SOC_est_1 모델 개요

### 주요 파라미터
\begin{align*}
\text{SOC}_0 &= \text{초기 SOC (0에서 1 사이)} \\
C_{\text{rated}} &= \text{정격 용량 [C]} \\
\eta_{\text{chg}} &= \text{충전 효율 (Coulombic efficiency)} \\
\eta_{\text{dis}} &= \text{방전 효율 (Coulombic efficiency)} \\
\text{dischargePositive} &= \text{전류 방향 설정 (true: I>0=방전)}
\end{align*}

### 입력/출력
- **입력**: $I$ (배터리 전류 [A])
- **출력**: $\text{SOC}_{\text{est}}$ (추정된 SOC [0..1])

### 기본 수식
쿨롱 계수법 기반 SOC 추정:
$$\text{SOC}(t) = \text{SOC}_0 + \int_0^t -\frac{I_{\text{eff}}(\tau)}{C_{\text{rated}}} d\tau$$

## 2. SOC_est_1 로직 플로우차트

### 전체 로직 흐름

```latex
\begin{tikzpicture}[node distance=2cm, auto]
    % 노드 스타일 정의
    \tikzstyle{process} = [rectangle, rounded corners, minimum width=3cm, minimum height=1cm, text centered, draw=black, fill=blue!10]
    \tikzstyle{decision} = [diamond, minimum width=3cm, minimum height=1cm, text centered, draw=black, fill=green!10]
    \tikzstyle{io} = [trapezium, trapezium left angle=70, trapezium right angle=110, minimum width=2cm, minimum height=1cm, text centered, draw=black, fill=yellow!10]
    \tikzstyle{arrow} = [thick,->,>=stealth]

    % 노드 배치
    \node (input) [io] {입력 전류 $I$ [A]};
    \node (sign) [process, below of=input] {signGain\\전류 방향 변환};
    \node (gezero) [decision, right of=sign, xshift=2cm] {geZero\\$I \geq 0$?};
    \node (proddis) [process, above of=gezero] {prodDis\\$I \times (1/\eta_{dis})$};
    \node (prodchg) [process, below of=gezero] {prodChg\\$I \times \eta_{chg}$};
    \node (pick) [process, right of=gezero, xshift=2cm] {pickIeff\\유효 전류 선택};
    \node (kgain) [process, below of=pick] {kGain\\$\frac{-I_{eff}}{C_{rated}}$};
    \node (integ) [process, below of=kgain] {integ\\SOC 적분};
    \node (limit) [process, below of=integ] {limiter\\범위 제한};
    \node (output) [io, below of=limit] {출력 $\text{SOC}_{est}$ [0..1]};

    % 화살표 연결
    \draw [arrow] (input) -- (sign);
    \draw [arrow] (sign) -- (proddis);
    \draw [arrow] (sign) -- (gezero);
    \draw [arrow] (sign) -- (prodchg);
    \draw [arrow] (proddis) -- (pick);
    \draw [arrow] (gezero) -- (pick);
    \draw [arrow] (prodchg) -- (pick);
    \draw [arrow] (pick) -- (kgain);
    \draw [arrow] (kgain) -- (integ);
    \draw [arrow] (integ) -- (limit);
    \draw [arrow] (limit) -- (output);

    % 추가 설명
    \node [text width=4cm, align=center, below of=gezero, yshift=-1cm] {방전 모드: prodDis\\충전 모드: prodChg};
\end{tikzpicture}
```

### 상세 수식 설명

1. **전류 방향 변환**:
   $$I' = \begin{cases}
   +I & \text{if dischargePositive = true} \\
   -I & \text{if dischargePositive = false}
   \end{cases}$$

2. **효율 적용**:
   $$\begin{cases}
   I_{\text{eff,dis}} = I' \times \frac{1}{\eta_{\text{dis}}} & \text{(방전 시)} \\
   I_{\text{eff,chg}} = I' \times \eta_{\text{chg}} & \text{(충전 시)}
   \end{cases}$$

3. **유효 전류 선택**:
   $$I_{\text{eff}} = \begin{cases}
   I_{\text{eff,dis}} & \text{if } I' \geq 0 \\
   I_{\text{eff,chg}} & \text{if } I' < 0
   \end{cases}$$

4. **SOC 변화율**:
   $$\frac{d\text{SOC}}{dt} = -\frac{I_{\text{eff}}}{C_{\text{rated}}}$$

5. **SOC 적분**:
   $$\text{SOC}(t) = \text{SOC}_0 + \int_0^t \frac{d\text{SOC}}{d\tau} d\tau$$

6. **범위 제한**:
   $$\text{SOC}_{\text{est}} = \max(0, \min(1, \text{SOC}(t)))$$

In [ ]:
## 3. Python 시뮬레이션 구현

```python
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint
import warnings
warnings.filterwarnings('ignore')

# SOC_est_1 파라미터 설정
SOC0 = 0.8              # 초기 SOC
C_rated = 200 * 3600    # 정격 용량 [C] (200Ah)
eta_chg = 0.99          # 충전 효율
eta_dis = 1.00          # 방전 효율
dischargePositive = True # I>0 = 방전

# 시뮬레이션 시간 설정
t = np.linspace(0, 3600, 3600)  # 1시간 시뮬레이션, 1초 간격

# 전류 프로파일 생성 (방전 → 휴지 → 충전)
def current_profile(t):
    if t < 1200:      # 0-20분: 50A 방전
        return 50.0
    elif t < 1800:    # 20-30분: 휴지
        return 0.0
    else:             # 30-60분: -30A 충전
        return -30.0

# SOC_est_1 로직 구현
def soc_estimator(y, t, I_func, SOC0, C_rated, eta_chg, eta_dis, dischargePositive):
    SOC = y[0]
    I = I_func(t)

    # 1. 전류 방향 변환 (signGain)
    if dischargePositive:
        I_prime = I      # I>0 = 방전
    else:
        I_prime = -I     # I>0 = 충전

    # 2. 효율 적용
    if I_prime >= 0:
        I_eff = I_prime * (1.0 / eta_dis)  # 방전
    else:
        I_eff = I_prime * eta_chg          # 충전

    # 3. SOC 변화율 계산 (kGain)
    dSOC_dt = -I_eff / C_rated

    return [dSOC_dt]

# 시뮬레이션 실행
I_values = [current_profile(ti) for ti in t]
y0 = [SOC0]  # 초기 SOC
sol = odeint(soc_estimator, y0, t, args=(current_profile, SOC0, C_rated, eta_chg, eta_dis, dischargePositive))
SOC_est = np.clip(sol[:, 0], 0, 1)  # 범위 제한 (limiter)

print("SOC_est_1 시뮬레이션 완료!")
print(".3f")
print(".3f")
print(".3f")

In [ ]:
## 4. 시뮬레이션 결과 시각화

```python
# 시각화
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8))

# 전류 프로파일
ax1.plot(t/60, I_values, 'b-', linewidth=2, label='배터리 전류 I [A]')
ax1.set_xlabel('시간 [분]')
ax1.set_ylabel('전류 [A]')
ax1.set_title('SOC_est_1 시뮬레이션: 전류 프로파일')
ax1.grid(True, alpha=0.3)
ax1.legend()
ax1.axhline(y=0, color='k', linestyle='--', alpha=0.5)

# SOC 추정 결과
ax2.plot(t/60, SOC_est, 'r-', linewidth=2, label='추정 SOC')
ax2.axhline(y=SOC0, color='g', linestyle='--', alpha=0.7, label=f'초기 SOC = {SOC0}')
ax2.axhline(y=1.0, color='orange', linestyle=':', alpha=0.5, label='SOC 최대값')
ax2.axhline(y=0.0, color='orange', linestyle=':', alpha=0.5, label='SOC 최소값')
ax2.fill_between(t/60, 0, 1, alpha=0.1, color='gray', label='유효 범위')
ax2.set_xlabel('시간 [분]')
ax2.set_ylabel('SOC [0-1]')
ax2.set_title('SOC_est_1 시뮬레이션: SOC 추정 결과')
ax2.grid(True, alpha=0.3)
ax2.legend()
ax2.set_ylim(-0.1, 1.1)

plt.tight_layout()
plt.show()

# 수치 결과 출력
print("\n=== 시뮬레이션 결과 분석 ===")
print(f"초기 SOC: {SOC0}")
print(f"최종 SOC: {SOC_est[-1]:.4f}")
print(f"SOC 변화량: {SOC_est[-1] - SOC0:.4f}")
print(f"총 방전량: {np.sum([max(0, I) for I in I_values]) / 3600:.2f} Ah")
print(f"총 충전량: {np.sum([max(0, -I) for I in I_values]) / 3600:.2f} Ah")

# 효율 효과 계산
total_discharge = np.sum([max(0, I) for I in I_values]) / 3600  # Ah
total_charge = np.sum([max(0, -I) for I in I_values]) / 3600    # Ah
expected_soc_change = (total_charge * eta_chg - total_discharge / eta_dis) / (C_rated / 3600)
actual_soc_change = SOC_est[-1] - SOC0

print(f"\n효율 보정 전 예상 SOC 변화: {(total_charge - total_discharge) / (C_rated / 3600):.4f}")
print(f"효율 보정 후 예상 SOC 변화: {expected_soc_change:.4f}")
print(f"실제 SOC 변화: {actual_soc_change:.4f}")
print(f"오차: {abs(actual_soc_change - expected_soc_change):.6f}")
```

## 5. 분석 결과 및 결론

### SOC_est_1의 특징

1. **쿨롱 계수법 기반**: 전류 적분을 통해 SOC를 추정하는 기본적인 방법
2. **효율 보정**: 충전/방전 효율을 고려한 정확도 향상
3. **범위 제한**: SOC를 물리적으로 가능한 0-1 범위로 제한
4. **유연한 전류 방향 설정**: `dischargePositive` 파라미터로 다양한 배터리 시스템 지원

### LaTeX 다이어그램의 장점

- **수학적 엄밀성**: 복잡한 수식을 명확하게 표현
- **시각적 명확성**: 블록 다이어그램으로 로직 흐름을 직관적으로 이해
- **재현성**: 동일한 다이어그램을 다른 문서에서도 사용 가능

### Python 시뮬레이션의 이점

- **실시간 검증**: Modelica 모델의 로직을 Python으로 검증
- **시각화**: matplotlib을 통한 직관적인 결과 표시
- **파라미터 스터디**: 다양한 조건에서 모델 동작 확인
- **디버깅**: 각 단계별 중간 결과 확인 가능

### 실제 적용 시 고려사항

1. **초기 SOC 정확도**: SOC₀의 정확도가 전체 추정 정확도에 큰 영향
2. **효율 파라미터**: 배터리 특성에 따른 η_chg, η_dis 값의 보정이 필요
3. **전류 측정 정확도**: 고정밀 전류 센서의 사용 권장
4. **온도 보정**: 온도에 따른 효율 변화를 고려한 추가 모델링 필요

### 확장 가능성

이 기본 모델은 다음과 같이 확장될 수 있습니다:
- **온도 보정**: SOC 추정에 온도 의존성 추가
- **열화 모델링**: 배터리 수명에 따른 용량 감소 고려
- **칼만 필터**: 노이즈가 있는 측정값에 대한 최적 추정
- **다중 모델**: SOC 레벨에 따른 다른 추정 알고리즘 적용

---
*이 노트북은 SOC_est_1.mo의 로직을 LaTeX 다이어그램과 Python 시뮬레이션으로 분석한 예시입니다.*